# Machine Learning to predict Down Jones Industrial Average

This simple Machine Learning example shows how to predict [^DJI value](https://finance.yahoo.com/quote/%5EDJI?p=^DJI&.tsrc=fin-srch) based on the past calculated averages.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1">Setup</a></span></li><li><span><a href="#Read-data-into-a-SFrame" data-toc-modified-id="Read-data-into-a-SFrame-2">Read data into a SFrame</a></span><ul class="toc-item"><li><span><a href="#TODO:-Value-should-be-original-value" data-toc-modified-id="TODO:-Value-should-be-original-value-2.1">TODO: Value should be original value</a></span></li></ul></li><li><span><a href="#Select-the-data-to-train-and-test" data-toc-modified-id="Select-the-data-to-train-and-test-3">Select the data to train and test</a></span><ul class="toc-item"><li><span><a href="#TODO:-Let's-NOT-take-last-few-days" data-toc-modified-id="TODO:-Let's-NOT-take-last-few-days-3.1">TODO: Let's NOT take last few days</a></span></li></ul></li><li><span><a href="#Create-the-model" data-toc-modified-id="Create-the-model-4">Create the model</a></span><ul class="toc-item"><li><span><a href="#Print-example-predictions" data-toc-modified-id="Print-example-predictions-4.1">Print example predictions</a></span></li></ul></li><li><span><a href="#&quot;Be-Less-Wrong&quot;" data-toc-modified-id="&quot;Be-Less-Wrong&quot;-5">"Be Less Wrong"</a></span><ul class="toc-item"><li><span><a href="#Previous-results:" data-toc-modified-id="Previous-results:-5.1">Previous results:</a></span><ul class="toc-item"><li><span><a href="#^DJI-averages-only" data-toc-modified-id="^DJI-averages-only-5.1.1">^DJI averages only</a></span></li></ul></li><li><span><a href="#TODO:-find-the-best-model" data-toc-modified-id="TODO:-find-the-best-model-5.2">TODO: find the best model</a></span></li></ul></li><li><span><a href="#Save-the-model" data-toc-modified-id="Save-the-model-6">Save the model</a></span></li></ul></div>

## Setup

In [1]:
# Install TuriCreate. Last updated November 4, 2020

# !pip install --upgrade pip
# !pip install Turicreate

In [2]:
import turicreate as tc

In [3]:
# Location of the spreadsheet (Comma Delimited Value) with ^DJI info that I prpared in a separate notebook.
data_path="./DATA/processed/^DJI.csv"

## Read data into a SFrame

In [4]:
# Load the data
data =  tc.SFrame(data_path)
data[363:370] # show data sample

Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/^DJI.csv

Parsing completed. Parsed 100 lines in 0.031337 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/^DJI.csv

Parsing completed. Parsed 13066 lines in 0.02337 secs.

Day,Date,Value,Original,Avg005,Avg030,Avg090,Avg180,Avg365
725033,1986-01-27,1548.170044,-125.0,1535.95,1545.43,1502.01,1420.96,0.0
725034,1986-01-28,1561.349976,-125.0,1544.34,1545.85,1503.97,1422.07,1361.29
725035,1986-01-29,1578.099976,-125.0,1552.48,1546.54,1506.14,1423.28,1362.07
725036,1986-01-30,1572.589966,-125.0,1559.52,1546.87,1508.12,1424.46,1362.8
725037,1986-01-31,1582.910034,-125.0,1568.62,1547.53,1510.23,1425.7,1363.6
725038,1986-02-01,1582.910034,-125.0,1575.57,1548.57,1512.33,1426.98,1364.41
725039,1986-02-02,1582.910034,-125.0,1579.88,1549.43,1514.38,1428.27,1365.22


### TODO: Value should be original value

Please note the the "High" is normalized to Int8, 
but for the prediciton purposes it should be an original "real" value.

## Select the data to train and test

In [5]:
row_count = len(data)
# Do not take initial year data as averages are not complete
data = data[365:row_count] 
# Make a train-test split
train_data, test_data = data.random_split(0.8)

### TODO: Let's NOT take last few days

I need to save the last few days to see if I can really predict upcoming values.

## Create the model

- https://apple.github.io/turicreate/docs/api/generated/turicreate.regression.create.html
- Automatically picks the right model based on your data.
- target: is the number to be predicted.
- features: are the the values that we ues to try to find pattern leading to prediciton.

In [6]:
model = tc.regression.create(
    train_data, 
    target='Original',
    features = [
        'Avg005',
        'Avg030',
        'Avg090',
        'Avg180',
        'Avg365'
    ],
    validation_set='auto', 
    verbose=True
)

Automatically generating validation set from 5% of the data.

Linear regression:

--------------------------------------------------------

Number of examples          : 9580

Number of features          : 5

...

| 3         | 0.026701     | 50.160095          | 43.951843            | 26.256676                       | 25.778711                         |

| 4         | 0.034125     | 37.904823          | 31.089729            | 18.423555                       | 18.062300                         |

| 5         | 0.040685     | 30.165230          | 22.268845            | 12.936590                       | 12.662284                         |

| 10        | 0.072574     | 12.793251          | 6.011383             | 2.377372                        | 2.327458                          |

+-----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+

# Predict values on data that was NOT used in training

In [7]:
#test_data.explore()
test_data

Day,Date,Value,Original,Avg005,Avg030,Avg090,Avg180,Avg365
725036,1986-01-30,1572.589966,-125.0,1559.52,1546.87,1508.12,1424.46,1362.8
725039,1986-02-02,1582.910034,-125.0,1579.88,1549.43,1514.38,1428.27,1365.22
725043,1986-02-06,1616.51001,-125.0,1602.57,1555.59,1523.22,1434.39,1368.65
725045,1986-02-08,1622.819946,-125.0,1615.09,1560.15,1527.94,1437.66,1370.44
725046,1986-02-09,1622.819946,-125.0,1617.36,1563.21,1530.06,1439.31,1371.33
725048,1986-02-11,1637.5,-125.0,1627.82,1570.17,1534.33,1442.76,1373.2
725050,1986-02-13,1649.310059,-124.0,1636.65,1577.91,1538.72,1446.35,1375.12
725057,1986-02-20,1675.459961,-124.0,1677.52,1608.33,1555.74,1459.91,1382.39
725060,1986-02-23,1702.75,-124.0,1694.59,1625.86,1563.41,1466.21,1385.81
725068,1986-03-03,1714.329956,-124.0,1728.19,1668.33,1584.89,1483.2,1395.12


In [8]:
## Save predictions to an SArray
predictions = model.predict(test_data)
#predictions

### Print example predictions

In [9]:
start = 0
end = len(predictions)
step = 50

for id in range(start, end, step):
    a = round( predictions[id], 2)
    b = test_data[id]["Original"]
    print( "predicted ", round(a, 0), "\t, but actual value was \t", round(b, 0) , "\t difference is \t", round(b-a, 2) ) # dict

predicted  -125.0 	, but actual value was 	 -125.0 	 difference is 	 0.21
predicted  -122.0 	, but actual value was 	 -122.0 	 difference is 	 0.07
predicted  -118.0 	, but actual value was 	 -119.0 	 difference is 	 -0.52
predicted  -122.0 	, but actual value was 	 -122.0 	 difference is 	 -0.15
predicted  -120.0 	, but actual value was 	 -120.0 	 difference is 	 -0.41
predicted  -114.0 	, but actual value was 	 -114.0 	 difference is 	 0.2
predicted  -115.0 	, but actual value was 	 -114.0 	 difference is 	 0.61
predicted  -117.0 	, but actual value was 	 -117.0 	 difference is 	 -0.49
predicted  -112.0 	, but actual value was 	 -112.0 	 difference is 	 -0.27
predicted  -110.0 	, but actual value was 	 -110.0 	 difference is 	 -0.11
predicted  -110.0 	, but actual value was 	 -110.0 	 difference is 	 -0.08
predicted  -107.0 	, but actual value was 	 -107.0 	 difference is 	 0.25
predicted  -105.0 	, but actual value was 	 -105.0 	 difference is 	 -0.44
predicted  -104.0 	, but actual

## "Be Less Wrong"

Evaluate how good is the model

It appears that the predition results vary from run to run so it is worth to run it until you find the model with minimum error, 

or **as Elon Musk says "Be less wrong"**.

### Previous results:

#### ^DJI averages only 

- {'max_error': 1749.5078773959249, 'rmse': 124.58897796835019}
- {'max_error': 1621.9227669335778, 'rmse': 106.39104997423203}

TODO: write this in a loop to select the best model

### TODO: find the best model

Create a "for" loop to find the best model

In [10]:
# Evaluate the model and save the results into a dictionary
results = model.evaluate( test_data ) #test_data[0:2531]
results

{'max_error': 8.845726392048448, 'rmse': 0.9326600626868612}

## Save the model

Save the model for future use in MacOS, iOS, etc. applications

In [11]:
# Export to Core ML
model.export_coreml('./DATA/models/^DJI.mlmodel')